In [77]:
import sys
sys.path.append("../")
import copy
import re
import pandas as pd
import numpy as np
from experiments import ExperimentModule
from experiments import (
    save_plots, 
    write_row, 
    write_opcounts,
    save_circuit,
    load_qasm_circuit,
    create_dir,
    get_random_state
)
import matplotlib.pyplot as plt
from itertools import product

%matplotlib inline

In [78]:
results_dir = "../results/COBYLA/no_ent/random_states"
run_range = list(range(1, 11))
#qubit_range = [3, 4, 5, 6, 7, 8]
qubit_range = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
#qubit_range = [3, 4, 5, 6, 7, 8, 9, 10]

# EPS values are mapped separately
#eps_values = [0.5, 0.1, 0.05, 0.01, 0.005, 0.001]
eps_values = [0.5, 0.1, 0.05, 0.01]

### Random State type

In [79]:
state_type = "random_state"

In [80]:
#[0.5, 0.1, 0.05, 0.01, 0.005, 0.001]
original_results_dict = {
    "num_qubits": [],
    0.5: [],
    0.1: [],
    0.05: [],
    0.01: [],
}

modified_results_dict = copy.deepcopy(original_results_dict)

for (r_dir, st_type, num_qubits, eps) in product([results_dir], [state_type], qubit_range, eps_values):
    if not num_qubits in original_results_dict["num_qubits"]: 
        original_results_dict["num_qubits"] += [num_qubits]
        modified_results_dict["num_qubits"] += [num_qubits]

    original_results_dict[eps] += [[]]
    modified_results_dict[eps] += [[]]

    for run_idx in run_range:
        fid_loss_original = np.loadtxt(f"{results_dir}/{st_type}/run_{run_idx}/csv/original_fidloss_{num_qubits}qb_{eps}eps.csv", delimiter=",")
        data_original = fid_loss_original[-1][-1] if len(fid_loss_original.shape) > 1 else fid_loss_original[-1]
        original_results_dict[eps][-1] += [data_original.round(4)]

        fid_loss_modified = np.loadtxt(f"{results_dir}/{st_type}/run_{run_idx}/csv/modified_fidloss_{num_qubits}qb_{eps}eps.csv", delimiter=",")
        data_modified = fid_loss_modified[-1][-1] if len(fid_loss_modified.shape) > 1 else fid_loss_modified[-1]
        modified_results_dict[eps][-1]+= [data_modified.round(4)]

In [81]:
default_dict = {
    "num_qubits": qubit_range,
    0.5: [],
    0.1: [],
    0.05: [],
    0.01: [],
}
mean_original_results_dict = copy.deepcopy(default_dict)
mean_modified_results_dict = copy.deepcopy(default_dict)

stds_original_results_dict = copy.deepcopy(default_dict)
stds_modified_results_dict = copy.deepcopy(default_dict)

for eps in eps_values:
    mean_original_results_dict[eps] = np.mean(original_results_dict[eps], axis=1)
    stds_original_results_dict[eps] = np.std(original_results_dict[eps], axis=1).round(2)

    mean_modified_results_dict[eps] = np.mean(modified_results_dict[eps], axis=1)
    stds_modified_results_dict[eps] = np.std(modified_results_dict[eps], axis=1).round(2)

Original fidloss

In [82]:
df_mean_original = pd.DataFrame(mean_original_results_dict)
df_mean_original = df_mean_original.set_index("num_qubits")
display(df_mean_original)

df_stds_original = pd.DataFrame(stds_original_results_dict)
df_stds_original = df_stds_original.set_index("num_qubits")
df_stds_original

,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.05950,0.05950,0.00133,0.00243
4,0.17780,0.17780,0.12383,0.00217
5,0.19080,0.19080,0.17740,0.15213
6,0.22480,0.22480,0.20020,0.17441
7,0.24860,0.24860,0.24340,0.22460
8,0.23640,0.23640,0.71764,0.22630
9,0.25290,0.25290,0.79041,0.70604
10,0.71120,0.71863,0.60187,0.89479
11,0.38078,0.38416,0.69303,0.61979


,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.03
6,0.00,0.00,0.00,0.00
7,0.00,0.00,0.00,0.00
8,0.00,0.00,0.04,0.00
9,0.00,0.00,0.03,0.03
10,0.04,0.05,0.16,0.01
11,0.04,0.08,0.05,0.05


In [83]:
print(df_mean_original.to_latex(float_format="%.4f"))

\begin{tabular}{lrrrr}
\toprule
 & 0.500000 & 0.100000 & 0.050000 & 0.010000 \\
num_qubits &  &  &  &  \\
\midrule
3 & 0.0595 & 0.0595 & 0.0013 & 0.0024 \\
4 & 0.1778 & 0.1778 & 0.1238 & 0.0022 \\
5 & 0.1908 & 0.1908 & 0.1774 & 0.1521 \\
6 & 0.2248 & 0.2248 & 0.2002 & 0.1744 \\
7 & 0.2486 & 0.2486 & 0.2434 & 0.2246 \\
8 & 0.2364 & 0.2364 & 0.7176 & 0.2263 \\
9 & 0.2529 & 0.2529 & 0.7904 & 0.7060 \\
10 & 0.7112 & 0.7186 & 0.6019 & 0.8948 \\
11 & 0.3808 & 0.3842 & 0.6930 & 0.6198 \\
12 & 0.9299 & 0.9361 & 0.6485 & 0.9024 \\
13 & 0.7467 & 0.7825 & 0.6651 & 0.9539 \\
14 & 0.9856 & 0.9855 & 0.9254 & 0.9710 \\
15 & 0.9142 & 0.9123 & 0.9533 & 0.9944 \\
\bottomrule
\end{tabular}



Modified fidloss

In [84]:
df_mean_modified = pd.DataFrame(mean_modified_results_dict)
df_mean_modified = df_mean_modified.set_index("num_qubits")
display(df_mean_modified)

df_stds_modified = pd.DataFrame(stds_modified_results_dict)
df_stds_modified = df_stds_modified.set_index("num_qubits")
df_stds_modified

,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.05950,0.05950,0.00867,0.00409
4,0.17420,0.17420,0.12410,0.00104
5,0.18270,0.18270,0.71420,0.14116
6,0.21140,0.21140,0.56654,0.71244
7,0.24200,0.24200,0.60720,0.21420
8,0.23310,0.23310,0.22610,0.93872
9,0.25090,0.25090,0.93121,0.62072
10,0.88780,0.88218,0.46609,0.99029
11,0.88977,0.89353,0.95334,0.25319


,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.00,0.00,0.01,0.01
4,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00
6,0.00,0.00,0.08,0.00
7,0.00,0.00,0.00,0.00
8,0.00,0.00,0.00,0.02
9,0.00,0.00,0.02,0.06
10,0.02,0.02,0.07,0.00
11,0.02,0.01,0.00,0.01


In [85]:
print(df_mean_modified.to_latex(float_format="%.4f"))

\begin{tabular}{lrrrr}
\toprule
 & 0.500000 & 0.100000 & 0.050000 & 0.010000 \\
num_qubits &  &  &  &  \\
\midrule
3 & 0.0595 & 0.0595 & 0.0087 & 0.0041 \\
4 & 0.1742 & 0.1742 & 0.1241 & 0.0010 \\
5 & 0.1827 & 0.1827 & 0.7142 & 0.1412 \\
6 & 0.2114 & 0.2114 & 0.5665 & 0.7124 \\
7 & 0.2420 & 0.2420 & 0.6072 & 0.2142 \\
8 & 0.2331 & 0.2331 & 0.2261 & 0.9387 \\
9 & 0.2509 & 0.2509 & 0.9312 & 0.6207 \\
10 & 0.8878 & 0.8822 & 0.4661 & 0.9903 \\
11 & 0.8898 & 0.8935 & 0.9533 & 0.2532 \\
12 & 0.8630 & 0.8676 & 0.9308 & 0.9851 \\
13 & 0.9426 & 0.9481 & 0.9760 & 0.9762 \\
14 & 0.9957 & 0.9949 & 0.7979 & 0.9927 \\
15 & 0.9880 & 0.9875 & 0.9995 & 0.9591 \\
\bottomrule
\end{tabular}



In [86]:
df_mean_original < df_mean_modified


,0.5,0.1,0.05,0.01
num_qubits,,,,
3,False,False,True,True
4,False,False,True,False
5,False,False,True,False
6,False,False,True,True
7,False,False,True,False
8,False,False,False,True
9,False,False,True,False
10,True,True,False,True
11,True,True,True,False


### Random Sparse States

In [87]:
state_type = "random_sparse_state"
density = 0.3

In [88]:
#[0.5, 0.1, 0.05, 0.01, 0.005, 0.001]
original_results_dict = {
    "num_qubits": [],
    0.5: [],
    0.1: [],
    0.05: [],
    0.01: [],
}

modified_results_dict = copy.deepcopy(original_results_dict)

for (r_dir, st_type, num_qubits, eps) in product([results_dir], [state_type], qubit_range, eps_values):
    if not num_qubits in original_results_dict["num_qubits"]: 
        original_results_dict["num_qubits"] += [num_qubits]
        modified_results_dict["num_qubits"] += [num_qubits]

    original_results_dict[eps] += [[]]
    modified_results_dict[eps] += [[]]

    for run_idx in run_range:
        fid_loss_original = np.loadtxt(f"{results_dir}/{st_type}/run_{run_idx}/density_{density}/csv/original_fidloss_{num_qubits}qb_{eps}eps.csv", delimiter=",")
        data_original = fid_loss_original[-1][-1] if len(fid_loss_original.shape) > 1 else fid_loss_original[-1]
        original_results_dict[eps][-1] += [data_original.round(4)]

        fid_loss_modified = np.loadtxt(f"{results_dir}/{st_type}/run_{run_idx}/density_{density}/csv/modified_fidloss_{num_qubits}qb_{eps}eps.csv", delimiter=",")
        data_modified = fid_loss_modified[-1][-1] if len(fid_loss_modified.shape) > 1 else fid_loss_modified[-1]
        modified_results_dict[eps][-1]+= [data_modified.round(4)]

In [89]:
default_dict = {
    "num_qubits": qubit_range,
    0.5: [],
    0.1: [],
    0.05: [],
    0.01: [],
}
mean_original_results_dict = copy.deepcopy(default_dict)
mean_modified_results_dict = copy.deepcopy(default_dict)

stds_original_results_dict = copy.deepcopy(default_dict)
stds_modified_results_dict = copy.deepcopy(default_dict)

for eps in eps_values:
    mean_original_results_dict[eps] = np.mean(original_results_dict[eps], axis=1)
    stds_original_results_dict[eps] = np.std(original_results_dict[eps], axis=1).round(2)

    mean_modified_results_dict[eps] = np.mean(modified_results_dict[eps], axis=1)
    stds_modified_results_dict[eps] = np.std(modified_results_dict[eps], axis=1).round(2)

In [90]:
df_mean_original = pd.DataFrame(mean_original_results_dict)
df_mean_original = df_mean_original.set_index("num_qubits")
display(df_mean_original)
display(df_mean_modified)

df_stds_original = pd.DataFrame(stds_original_results_dict)
df_stds_original = df_stds_original.set_index("num_qubits")
#df_stds_original

,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.93394,0.86036,0.00002,0.00000
4,0.91560,0.87726,0.90536,0.00000
5,0.51677,0.51650,0.42062,0.32242
6,0.65681,0.65657,0.62846,0.52643
7,0.65540,0.67824,0.73202,0.59948
8,0.73720,0.73720,0.78863,0.71936
9,0.75630,0.77757,0.93519,0.83812
10,0.77177,0.77174,0.77298,0.94515
11,0.79644,0.77528,0.79348,0.84554


,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.05950,0.05950,0.00867,0.00409
4,0.17420,0.17420,0.12410,0.00104
5,0.18270,0.18270,0.71420,0.14116
6,0.21140,0.21140,0.56654,0.71244
7,0.24200,0.24200,0.60720,0.21420
8,0.23310,0.23310,0.22610,0.93872
9,0.25090,0.25090,0.93121,0.62072
10,0.88780,0.88218,0.46609,0.99029
11,0.88977,0.89353,0.95334,0.25319


In [91]:
display(df_stds_original)
display(df_stds_modified)

,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.08,0.21,0.00,0.00
4,0.06,0.10,0.09,0.00
5,0.00,0.00,0.00,0.01
6,0.00,0.00,0.05,0.01
7,0.00,0.07,0.03,0.00
8,0.00,0.00,0.07,0.03
9,0.00,0.06,0.01,0.02
10,0.00,0.00,0.00,0.01
11,0.06,0.00,0.02,0.02


,0.5,0.1,0.05,0.01
num_qubits,,,,
3,0.00,0.00,0.01,0.01
4,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00
6,0.00,0.00,0.08,0.00
7,0.00,0.00,0.00,0.00
8,0.00,0.00,0.00,0.02
9,0.00,0.00,0.02,0.06
10,0.02,0.02,0.07,0.00
11,0.02,0.01,0.00,0.01


In [92]:
print("--"*100)
print("Original Mean")
print("--"*100)
print(df_mean_original.to_latex(float_format="%.4f"))

print("--"*100)
print("Modified Mean")
print("--"*100)
print(df_mean_modified.to_latex(float_format="%.4f"))

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Original Mean
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
\begin{tabular}{lrrrr}
\toprule
 & 0.500000 & 0.100000 & 0.050000 & 0.010000 \\
num_qubits &  &  &  &  \\
\midrule
3 & 0.9339 & 0.8604 & 0.0000 & 0.0000 \\
4 & 0.9156 & 0.8773 & 0.9054 & 0.0000 \\
5 & 0.5168 & 0.5165 & 0.4206 & 0.3224 \\
6 & 0.6568 & 0.6566 & 0.6285 & 0.5264 \\
7 & 0.6554 & 0.6782 & 0.7320 & 0.5995 \\
8 & 0.7372 & 0.7372 & 0.7886 & 0.7194 \\
9 & 0.7563 & 0.7776 & 0.9352 & 0.8381 \\
10 & 0.7718 & 0.7717 & 0.7730 & 0.9452 \\
11 & 0.7964 & 0.7753 & 0.7935 & 0.8455 \\
12 & 0.9468 & 0.9430 & 0.8827 & 0.9760 \\
13 & 0.9196 & 0.9045 & 0.9116 & 0.9667 \\
14 & 0.9964 & 

In [93]:
df_mean_original < df_mean_modified

,0.5,0.1,0.05,0.01
num_qubits,,,,
3,False,False,True,True
4,False,False,False,True
5,False,False,True,False
6,False,False,False,True
7,False,False,False,False
8,False,False,False,True
9,False,False,False,False
10,True,True,False,True
11,True,True,True,False
